<a href="https://colab.research.google.com/github/yaolihua081/AIG_PC/blob/main/week4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import sys, matplotlib.pylab as plt, re, platform, matplotlib, gensim
import numpy as np, pandas as pd, nltk, sklearn
tmp = nltk.download(['stopwords', 'gutenberg','punkt'], quiet=True) # See https://www.nltk.org/book/ch02.html
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
stopword_list = nltk.corpus.stopwords.words('english')

# Increase viewable area of Pandas tables, numpy arrays, plots
pd.set_option('max_rows', 5, 'max_columns', 500, 'max_colwidth', 1, 'precision', 2)
np.set_printoptions(linewidth=10000, precision=4, edgeitems=20, suppress=True)
plt.rcParams['figure.figsize'] = [16, 4]

# Check your package versions against those used in generating the answers: 
print( 'Grader:Py/3.6.9, skl/0.22.2.post1, pd/1.0.5, np/1.18.5, plt/3.2.2, nltk/3.2.5, gensim/3.6.0')
print(f'Yours: Py/{platform.python_version()}, skl/{sklearn.__version__}'\
      f', pd/{pd.__version__}, np/{np.__version__}, plt/{matplotlib.__version__}'\
      f', nltk/{nltk.__version__}, gensim/{gensim.__version__}')

# small dictionary of strings (or sentences)
dosSmall = dict(s1="I like Python.", s2="Python & R r Gr8!", s3="There is no R in Python.",
                s4="Python to R is like apples to oranges.", s5="I like apples and oranges.", 
                s6="Oranges and apples are sour.", s7="Apples r no oranges.")


Grader:Py/3.6.9, skl/0.22.2.post1, pd/1.0.5, np/1.18.5, plt/3.2.2, nltk/3.2.5, gensim/3.6.0
Yours: Py/3.6.9, skl/0.22.2.post1, pd/1.0.5, np/1.18.5, plt/3.2.2, nltk/3.2.5, gensim/3.6.0


In [ ]:
dosSmall.values()

dict_values(['I like Python.', 'Python & R r Gr8!', 'There is no R in Python.', 'Python to R is like apples to oranges.', 'I like apples and oranges.', 'Oranges and apples are sour.', 'Apples r no oranges.'])

In [ ]:
#Q1 correct, 0.57
#cvSmall = CountVectorizer(stop_words=stopword_list)
#cvSmall = CountVectorizer(stop_words=stopword_list,token_pattern=r'(?u)\b\w\w+\b') #0.59
#keep only one letter word
#cvSmall = CountVectorizer(stop_words=stopword_list,token_pattern=r'(?u)\b[\w]{1,}\b') #0.57
cvSmall = CountVectorizer(stop_words=stopword_list,token_pattern=r'(?u)\b\w\w*\b') #0.57
smDT= cvSmall.fit_transform(dosSmall.values())

mDT=smDT.toarray()

mDT
print(cvSmall.get_feature_names())
#document term matrix
filter = mDT == 0
sum(filter)
filter 
TotZero=sum(filter).sum()
# for i in len(mDT[0])

Answer=TotZero/(mDT.size)
Answer
print('. ')
mDT.size 

array([[0, 0, 1, 0, 1, 0, 0],
       [0, 1, 0, 0, 1, 2, 0],
       [0, 0, 0, 0, 1, 1, 0],
       [1, 0, 1, 1, 1, 1, 0],
       [1, 0, 1, 1, 0, 0, 0],
       [1, 0, 0, 1, 0, 0, 1],
       [1, 0, 0, 1, 0, 1, 0]])

['apples', 'gr8', 'like', 'oranges', 'python', 'r', 'sour']


array([3, 6, 4, 3, 3, 3, 6])

array([[ True,  True, False,  True, False,  True,  True],
       [ True, False,  True,  True, False, False,  True],
       [ True,  True,  True,  True, False, False,  True],
       [False,  True, False, False, False, False,  True],
       [False,  True, False, False,  True,  True,  True],
       [False,  True,  True, False,  True,  True, False],
       [False,  True,  True, False,  True, False,  True]])

0.5714285714285714

. 


49

In [ ]:
#Q2 correct,11
#fWS = ... # := frequency for word 'r' in sentence 's2'
#fS = ... # := sum of frequences for sentence s4
#fW1 = ... # := sum of frequences for word 'python'
#fW2 = ... # := sum of frequences for word 'there'
#mDT
# it was 8 before 
#now it is 11
#column_names=['s1','s2','s3','s4','s5','s6','s7']
row_names=dosSmall.keys()
column_names=cvSmall.get_feature_names()
df = pd.DataFrame(mDT, columns=column_names, index=row_names)
df
#df.loc['s2']
#df.loc['s2','python']
filter='r' in df.columns
fWS= df.loc['s2','r'] if filter  else 0

filter='there' in df.columns
fW2= sum(df['there']) if filter  else 0

fS=df[df.index=='s4'].sum().sum()
fW1=sum(df['python'])
fWS+fS+fW1+fW2
[fWS,fS,fW1,fW2]

,apples,gr8,like,oranges,python,r,sour
s1,0,0,1,0,1,0,0
s2,0,1,0,0,1,2,0
...,...,...,...,...,...,...,...
s6,1,0,0,1,0,0,1
s7,1,0,0,1,0,1,0


11

[2, 5, 4, 0]

In [ ]:
#Q3 correct 10
#c1 = ...     # := greatest similarity coefficient among different sentences
#c2 = ...     # := greatest similarity coefficient among sentences with themselves
#c3 = ...     # := similarity coefficient among sentences s1 and s2

similarityMatrix=df.dot(df.T)
#similarityMatrix.idxmax()
#similarityMatrix.iloc[0,3]
#similarityMatrix['s4'].value_counts().nlargest(n=1)
#c1=similarityMatrix.idxmax()
#c1
#c2=similarityMatrix.max().max()

c2=max(np.diag(similarityMatrix))
#c3=similarityMatrix.iloc[0,1]
c3=similarityMatrix['s1']['s2']
UpTrangle=np.triu(similarityMatrix, 1)
c1=UpTrangle.max()
#similarityMatrix.index(c1)
c1+c2+c3
[c1 ,c2,c3]
similarityMatrix
#s3 and s4 more similar

10

[3, 6, 1]

,s1,s2,s3,s4,s5,s6,s7
s1,2,1,1,2,1,0,0
s2,1,6,3,3,0,0,2
...,...,...,...,...,...,...,...
s6,0,0,0,2,2,3,2
s7,0,2,1,3,2,2,3


In [ ]:
#Q4, correct, 0.57
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
mTI=tfidf_transformer.fit_transform(mDT)
#mTI=tfidf_transformer.fit(smDT)
#mTI=tfidf_transformer.(smDT)
#mTI.toarray()
mTI=mTI.toarray()
df_idf = pd.DataFrame(mTI, columns=cvSmall.get_feature_names())
df_idf
filter = df_idf == 0
answer=np.sum(filter).sum()/df_idf.size
answer#0.57

,apples,gr8,like,oranges,python,r,sour
0,0.00,0.00,0.76,0.00,0.66,0.00,0.00
1,0.00,0.59,0.00,0.00,0.36,0.72,0.00
...,...,...,...,...,...,...,...
5,0.46,0.00,0.00,0.46,0.00,0.00,0.75
6,0.58,0.00,0.00,0.58,0.00,0.58,0.00


0.5714285714285714

In [ ]:
#Q5, correct 5.11
row_names=dosSmall.keys()
column_names=cvSmall.get_feature_names()
df = pd.DataFrame(mTI, columns=column_names, index=row_names)
df
filter='r' in df.columns
fWS= df.loc['s2','r'] if filter  else 0


filter='there' in df.columns
fW2= sum(df['there']) if filter  else 0

fS=df[df.index=='s4'].sum().sum()
fW1=sum(df['python'])
fWS+fS+fW1+fW2

[fWS,fS,fW1,fW2]

,apples,gr8,like,oranges,python,r,sour
s1,0.00,0.00,0.76,0.00,0.66,0.00,0.00
s2,0.00,0.59,0.00,0.00,0.36,0.72,0.00
...,...,...,...,...,...,...,...
s6,0.46,0.00,0.00,0.46,0.00,0.00,0.75
s7,0.58,0.00,0.00,0.58,0.00,0.58,0.00


5.113885818732456

[0.723802693786046, 2.2321954022359787, 2.157887722710431, 0]

In [ ]:
#Q6 
#correct, 2.03
#Q3
#c1 = ...     # := greatest similarity coefficient among different sentences
#c2 = ...     # := greatest similarity coefficient among sentences with themselves
#c3 = ...     # := similarity coefficient among sentences s1 and s2

similarityMatrix=df.dot(df.T)
similarityMatrix
c2=max(np.diag(similarityMatrix))
#c3=similarityMatrix['s1']['s2']
c3=similarityMatrix['s1']['s2']
UpTrangle=np.triu(similarityMatrix, 1)
c1=UpTrangle.max()
#similarityMatrix.index(c1)
c1+c2+c3
[c1 ,c2,c3]

,s1,s2,s3,s4,s5,s6,s7
s1,1.00,0.24,0.46,0.66,0.48,0.00,0.00
s2,0.24,1.00,0.77,0.47,0.00,0.00,0.42
...,...,...,...,...,...,...,...
s6,0.00,0.00,0.00,0.40,0.51,1.00,0.54
s7,0.00,0.42,0.41,0.75,0.63,0.54,1.00


2.0275319338211366

[0.790271270303694, 1.0000000000000002, 0.23726066351744238]

In [ ]:
#Q7,correct, 1.5

#c1 = ...     # := greatest similarity coefficient among different sentences
#c2 = ...     # := greatest similarity coefficient among sentences with themselves
#c3 = ...     # := similarity coefficient among sentences s1 and s2

#similarityMatrix=df.dot(df.T)

corrematrix=np.corrcoef(df)
c3=corrematrix[0,1]
c2=corrematrix.max()
UpTrangle=np.triu(corrematrix, 1)
c1=UpTrangle.max()
#similarityMatrix.index(c1)
c1+c2+c3
[c1 ,c2,c3]

corrematrix

1.503806628153322

[0.6563028532861449, 1.0, -0.15249622513282293]

array([[ 1.    , -0.1525,  0.2498,  0.4647,  0.2007, -0.5196, -0.5458],
       [-0.1525,  1.    ,  0.6563, -0.1518, -0.7042, -0.673 ,  0.0066],
       [ 0.2498,  0.6563,  1.    ,  0.3566, -0.5455, -0.5214,  0.0913],
       [ 0.4647, -0.1518,  0.3566,  1.    ,  0.5885, -0.3239,  0.4883],
       [ 0.2007, -0.7042, -0.5455,  0.5885,  1.    ,  0.1606,  0.359 ],
       [-0.5196, -0.673 , -0.5214, -0.3239,  0.1606,  1.    ,  0.2056],
       [-0.5458,  0.0066,  0.0913,  0.4883,  0.359 ,  0.2056,  1.    ]])

In [ ]:
#Q8 correct,2.03

normalized_df=df/np.linalg.norm(df,axis=1)
normalized_df
similarityMatrix=normalized_df.dot(normalized_df.T)
similarityMatrix
c2=max(np.diag(similarityMatrix))
c3=similarityMatrix['s1']['s2']
UpTrangle=np.triu(similarityMatrix, 1)
c1=UpTrangle.max()
#similarityMatrix.index(c1)
c1+c2+c3
[c1 ,c2,c3]



,apples,gr8,like,oranges,python,r,sour
s1,0.00,0.00,0.76,0.00,0.66,0.00,0.00
s2,0.00,0.59,0.00,0.00,0.36,0.72,0.00
...,...,...,...,...,...,...,...
s6,0.46,0.00,0.00,0.46,0.00,0.00,0.75
s7,0.58,0.00,0.00,0.58,0.00,0.58,0.00


,s1,s2,s3,s4,s5,s6,s7
s1,1.00,0.24,0.46,0.66,0.48,0.00,0.00
s2,0.24,1.00,0.77,0.47,0.00,0.00,0.42
...,...,...,...,...,...,...,...
s6,0.00,0.00,0.00,0.40,0.51,1.00,0.54
s7,0.00,0.42,0.41,0.75,0.63,0.54,1.00


2.0275319338211366

[0.790271270303694, 1.0000000000000002, 0.23726066351744246]

In [ ]:
#Q9
import gensim.downloader as api
wvMap = api.load("glove-wiki-gigaword-100") # load word vector dictionary: word-in => vector-out
#wvMap.vector_size

[==================================================] 100.0% 128.1/128.1MB downloaded


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
#Q9 correct,monty,dog,microsoft,yellow,red,coffee,princess,lucy

from collections import OrderedDict

myWords = ['python','cat','apple','orange','yellow','tea','queen','alice']

def finSimilar(wds):
  simlarWord=[]
  for wd in wds:
    catSimilar=wvMap.most_similar(wd) # a list, with pairs that have names and their similarity coefficinet with cat.
    top1=list(dict(catSimilar).keys())[0]
    simlarWord.append(top1)
  return simlarWord  

answer=finSimilar(myWords)
answerstr=','.join(answer)
answerstr
#monty,dog,microsoft,yellow,red,coffee,princess,lucy
#answer=[word for word in set(top1).intersection(set(myWords))]

#intersection(set(myWords))

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'monty,dog,microsoft,yellow,red,coffee,princess,lucy'

In [ ]:
#most_similar("cat") 

#top1=wvMap.most_similar("cat")


from collections import OrderedDict

myWords = ['python','cat','apple','orange','yellow','tea','queen','alice']

catSimilar=wvMap.most_similar("cat") # a list, with pairs that have names and their similarity coefficinet with cat.
dict(catSimilar).keys()
#type(top1) a list
#top2=dict(catSimilar)
#top2
#top2=sorted(top1.index())

#top3=OrderedDict(sorted(top2.items(), key=lambda t: t[1],reverse=True))
#top3

def getTopSimi(datlist):
  dictset=dict(datlist)
  orderdictset=OrderedDict(sorted(dictset.items(), key=lambda t: t[1],reverse=True))
  return orderdictset

#top1=getTopSimi(catSimilar)

#top1[list(top1.keys())[0]]
#top2
#wvMap["cat"] it is a list of  100-dimensional numbers
#wvMap.most_similar("cat").pop()

#To submit: a string of comma-separated top1 words for each token in myWords

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


dict_keys(['dog', 'rabbit', 'cats', 'monkey', 'pet', 'dogs', 'mouse', 'puppy', 'rat', 'spider'])

In [ ]:
#Q10,correct,20.83

def dotSimilar(wd1, wd2):
  vecApple=wvMap[wd1]
  vecOrange=wvMap[wd2]
  Apple_Orange_dot=vecApple.dot(vecOrange) 
  return Apple_Orange_dot

def CosinSimilar(wd1, wd2):
  vecApple=wvMap[wd1]
  vecOrange=wvMap[wd2]
  Apple_Orange_dot=vecApple.dot(vecOrange) /(np.linalg.norm(vecApple) * np.linalg.norm(vecOrange))
  return Apple_Orange_dot

c11=dotSimilar('apple','orange') 
c21=dotSimilar('apple','python') 
c12=CosinSimilar('apple','orange') 
c22=CosinSimilar('apple','python') 
c=[c11,c21,c12,c22]
c
sum(c)

#20.83

[16.628057, 3.5983427, 0.50070393, 0.105654016]

20.832758083939552

In [ ]:
#Q11 correct, said
search_phrases = ['Hatter speaks about tea and how it all began']
losAlice = [' '.join(s) for s in nltk.corpus.gutenberg.sents('carroll-alice.txt')]
losAlice = search_phrases + losAlice
cvSmall = CountVectorizer(stop_words=stopword_list)
smDT= cvSmall.fit_transform(losAlice)
mDT=smDT.toarray()
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
mTI=tfidf_transformer.fit_transform(smDT)
mTI=mTI.toarray()
row_names=losAlice
column_names=cvSmall.get_feature_names()

df = pd.DataFrame(mTI, columns=column_names,index=row_names)
mTI2=df.sum(axis=0) #fro each column, sum over the rows
maxElement = np.amax(mTI2)
result = np.where(mTI2 == maxElement)
print('Returned tuple of arrays :', result)
print('List of Indices of maximum element :', result[0])

maxElement #at 465 'curtsey', 2008, stick
#cvSmall.get_feature_names()[465]
column_names[1753]#'said'

df = pd.DataFrame(mTI, columns=column_names, index=row_names)
temp=df.sum(axis=0).sort_values(ascending=False).index[0]
temp


Returned tuple of arrays : (array([1753]),)
List of Indices of maximum element : [1753]


96.1773111535503

'said'

'said'

In [ ]:
temp=df.sum(axis=0).sort_values(ascending=False).index[1]
temp

'alice'

In [ ]:
#Q12 correct, 1521
#1521
#the index of the sentence in corpus most cosine-similar to our search phrase.
search_phrases = ['Hatter speaks about tea and how it all began']
#search_phrases = ['It is a wonderful book']
losAlice = [' '.join(s) for s in nltk.corpus.gutenberg.sents('carroll-alice.txt')]
losAlice = search_phrases + losAlice
cvSmall = CountVectorizer(stop_words=stopword_list)
smDT= cvSmall.fit_transform(losAlice)
#mDT=smDT.toarray()
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
mTI=tfidf_transformer.fit_transform(smDT)
mTI=mTI.toarray()
row_names=losAlice
column_names=cvSmall.get_feature_names()
df = pd.DataFrame(mTI, columns=column_names, index=row_names)

#normalized_df=df/np.linalg.norm(df,axis=1)
similarityMatrix=df.dot(df.T)/(np.linalg.norm(df,axis=1)*np.linalg.norm(df,axis=1))
#similarityMatri=df.dot(df.T)
#similarityMatrix=normalized_df.dot(normalized_df.T)
#/(np.linalg.norm(df,axis=0)*np.linalg.norm(df,axis=0))
#axis=1, norm over rows for each colums
#axis=1, for each rwo, normilize for the colums----each colums is dimension


#similarityMatrix.columns
#similarityMatrix.iloc[0,1]
maxElement = np.amax(similarityMatrix.iloc[0][1:])
result = list(np.where(similarityMatrix.iloc[0] == maxElement))
maxElement #203.55
type(result)#1518
print(result[0][0])
similarityMatrix.iloc[0,1521]
losAlice[result[0][0]]

#below also works
matrix_cosim_mTI_los_alice = cosine_similarity(mTI)

df = pd.DataFrame(matrix_cosim_mTI_los_alice)
temp=df[0].sort_values(ascending=False).index[1]
temp

#similarityMatrix

0.6214457545449842

list

1521


0.6214457545449842

"' It began with the tea ,' the Hatter replied ."

1521

In [ ]:
#Q test linkage
from scipy.cluster.hierarchy import dendrogram, linkage
similarity_matrix = cosine_similarity(mTI)

Z = linkage(similarity_matrix, 'ward')
pd.DataFrame(Z, columns=['Document\Cluster 1', 'Document\Cluster 2',
                         'Distance', 'Cluster Size'], dtype='object')


,Document\Cluster 1,Document\Cluster 2,Distance,Cluster Size
0,20,36,0,2
1,69,1.7e+03,0,3
...,...,...,...,...
1701,1.8e+03,3.4e+03,26,5.3e+02
1702,3.4e+03,3.4e+03,31,1.7e+03


In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=3, max_iter=10000, random_state=0)
dt_matrix = lda.fit_transform(mTI)
features = pd.DataFrame(dt_matrix, columns=['T1', 'T2', 'T3'])
features

KeyboardInterrupt: ignored

In [ ]:
#wvMap.vector_size
#len(wvMap.vocab.keys())
word2vec_vocabulary=wvMap.vocab.keys()

400000

In [ ]:
#Q13 correct, 0.98
#Use words from CountVectorizer and words from word2vec vocabulary in this exercise.
#To submit: fraction of words in corpus that are in word2vec
#import gensim.downloader as api
#wvMap = api.load("glove-wiki-gigaword-100")
losAlice = [' '.join(s) for s in nltk.corpus.gutenberg.sents('carroll-alice.txt')]
#cvSmall = CountVectorizer(stop_words=stopword_list,token_pattern=r'(?u)\b[\w]{1,}\b')
cvSmall = CountVectorizer(stop_words=stopword_list)
smDT= cvSmall.fit_transform(losAlice)
word2vecc_vabulary=wvMap.vocab.keys()
Alice_vocabulary=cvSmall.get_feature_names()
answer=len(set(Alice_vocabulary).intersection(set(word2vecc_vabulary)))/len(set(Alice_vocabulary))
answer

#len(set(Alice_vocabulary))
#len(set(word2vecc_vabulary))

0.9880165289256199

In [ ]:
#Q14,correct!
#cat,dog,cherry,red,red,drink,king,ann
#top1=wvMap.most_similar('python')
#dict(top1).keys()
A_voc=list(set(Alice_vocabulary).intersection(set(word2vecc_vabulary)))
myWords = ['python','cat','apple','orange','yellow','tea','queen','alice']
#return the first in a list that is in a set
def FirstWd(wds, Simlist):
  for wd in Simlist:
    if wd in wds:
      return wd      

def finSimilar(wds):
  simlarWord=[]
  for wd in wds:
    catSimilar=wvMap.most_similar(wd, topn=len(word2vecc_vabulary)) # a list, with pairs that have names and their similarity coefficinet with cat.
    top1=list(dict(catSimilar).keys())
    top=FirstWd(A_voc, top1)
    simlarWord.append(top)
  return simlarWord  

answer=finSimilar(myWords)
answer
answerstr=','.join(answer)
answerstr


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


['cat', 'dog', 'cherry', 'red', 'red', 'drink', 'king', 'ann']

'cat,dog,cherry,red,red,drink,king,ann'

In [ ]:
#Q14
#For all words in CountVectorer's vocab, compute cosine similarities 
#with the word "python" and choose the top1 word (that Alice would know in her world).
#wvMap = api.load("glove-wiki-gigaword-100") 
#Alice_vocabulary=cvSmall.get_feature_names()
word2vecc_vabulary=wvMap.vocab.keys()
myWords = ['python','cat','apple','orange','yellow','tea','queen','alice']
def CosSimilar(wds,wd1):
  CosSim=0
  SimMaxWord=''
  vecOrange=wvMap[wd1]
  for wd in wds:
    vecApple=wvMap[wd]
    Apple_Orange_dot=vecApple.dot(vecOrange) /(np.linalg.norm(vecApple)*np.linalg.norm(vecOrange))
    if CosSim <= Apple_Orange_dot:
      CosSim=Apple_Orange_dot
      SimMaxWord=wd
      #print([SimMaxWord,CosSim ] )
  return [SimMaxWord,CosSim ] 

A_voc=list(set(Alice_vocabulary).intersection(set(word2vecc_vabulary)))

def CosSimilarMyWords(wds1,wds2):
  toplist=[]
  corr=[]
  for wd in wds2:
    answer=CosSimilar(wds1,wd)   
    toplist.append(answer[0])
    corr.append(answer[1])

  return [toplist,corr] 
#myWords=['happy','peace','quite']
answer=CosSimilarMyWords(A_voc,myWords)
answer
answerstr=','.join(answer[0])
answerstr

CosSimilar(A_voc,'python')

[['cat', 'cat', 'apple', 'orange', 'red', 'tea', 'queen', 'alice'],
 [0.41910973, 1.0, 1.0, 1.0, 0.8597684, 1.0, 1.0, 1.0]]

'cat,cat,apple,orange,red,tea,queen,alice'

['cat', 0.41910973]

In [ ]:
cosine_similarity([wvMap['apple'], wvMap['apple']])


array([[1., 1.],
       [1., 1.]], dtype=float32)

In [ ]:
mDT_np = np.array(mDT)
mDT_np

array([[0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0],
       [1, 0, 1, 1, 1, 0],
       [1, 0, 1, 1, 0, 0],
       [1, 0, 0, 1, 0, 1],
       [1, 0, 0, 1, 0, 0]])

In [ ]:
filter = mDT_np == 0
filter
# sum(mDT[filter])
sum(filter).sum()

array([[ True,  True, False,  True, False,  True],
       [ True, False,  True,  True, False,  True],
       [ True,  True,  True,  True, False,  True],
       [False,  True, False, False, False,  True],
       [False,  True, False, False,  True,  True],
       [False,  True,  True, False,  True, False],
       [False,  True,  True, False,  True,  True]])

25

In [ ]:
x = np.matrix([[1, 2,3], [4, 3,6]])
norms=np.linalg.norm(x,axis=1)

x/norms[:,None]

matrix([[0.2673, 0.5345, 0.8018],
        [0.5121, 0.3841, 0.7682]])